In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import pandas as pd
import json
import folium
import geojson
import requests

In [ ]:
#putting slc and north area zips into arrays
slco_np = np.array([[20200416,84006,0],[20200416,84009,74.4],[20200416,84020,78.1],[20200416,84044,142.1],[20200416,84047,80.5],[20200416,84065,104.8],[20200416,84070,77.6],[20200416,84081,71.2],[20200416,84084,59.9],[20200416,84088,90.8],[20200416,84092,57.5],[20200416,84093,76.6],[20200416,84094,31.0],[20200416,84095,35.6],[20200416,84096,52.0],[20200416,84101,357.1],[20200416,84102,210.0],[20200416,84103,105.0],[20200416,84104,222.9],[20200416,84105,160.2],[20200416,84106,160.9],[20200416,84107,161.9],[20200416,84108,62.4],[20200416,84109,159.5],[20200416,84111,173.1],[20200416,84112,50.3],[20200416,84115,141],[20200416,84116,196.9],[20200416,84117,107.0],[20200416,84118,116.6],[20200416,84119,256.0],[20200416,84120,151.6],[20200416,84121,66.3],[20200416,84123,95.3],[20200416,84124,71.7],[20200416,84128,148.9],[20200416,84129,85.8]])
slco = pd.DataFrame(slco_np,columns = ['date','zip','rate per 100K'])
slco.head()
dv = pd.DataFrame(np.array([[20201416,84056,18.19,'Hill'],[20201416,84310,18.19,'Weber'],[20201416,84317,18.19,'Weber'],[20201416,84067,18.19,'Weber'],[20201416,84403,18.19,'Weber'],[20201416,84401,18.19,'Weber'],[20201416,84414,18.19,'Weber'],[20201416,84404,18.19,'Weber'],[20201416,84315,34.67,'Hooper'],[20201416,84015,34.67,'Hooper-Clearfield'],[20201416,84075,61.22,'Syracuse'],[20201416,84040,46.19,'Layton'],[20201416,84041,46.19,'Layton2'],[20201416,84405,46.19,'Layton-South Weber'],[20201416,84037,38.38,'Kaysville'],[20201416,84025,49.36,'Farmington'],[20201416,84014,96.90,'Centerville'],[20201416,84010,75.11,'Woods Cross'],[20201416,84054,109.41,'N Salt Lake'],[20201416,84025,114.94,'Bountiful'],[20201416,84087,75.1,'West Bountiful']]),columns = ['date','zip','rate per 100K','city'])
dv.head()

In [ ]:
slco.shape
nan_data = np.empty((37,1))
nan_data[:]=np.nan
slco['city']=nan_data
slco.head()
dv.head()

In [ ]:
dv['date'][:]= 20200416

In [ ]:
dv

In [ ]:
dv['date'] = pd.to_datetime(dv['date'], format = '%Y%m%d')

In [ ]:
all_df = dv.append(slco, ignore_index = True)
all_df.tail()
all_df.shape
#c17.head()

In [ ]:
zips_df = np.array(slco[['zip','rate per 100K']].append(dv[['zip','rate per 100K']]))

In [ ]:
zips_df = np.array(dv[['zip','rate per 100K']])

In [ ]:
zdf = pd.DataFrame(zips_df,columns=('zip','rate'))
#zdf.append(cv17['zip','rate'], ignore_index=True)

In [ ]:
type(zdf.zip[0])
type(zdf.rate[0])
zdf = zdf.astype({"zip": int, "rate": float})
type(zdf.zip[0])
type(zdf.rate[0])
zdf = zdf.astype({"zip": str, "rate": float})
type(zdf.zip[0])
type(zdf.rate[0])

In [ ]:
zdf.to_csv('zdf.csv')

In [ ]:
pd.DataFrame(zips_df).to_csv("zips.csv")

In [ ]:
pd.DataFrame(dv).to_csv('dv.csv')

In [ ]:
# request zip boundary data
req = requests.get('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ut_utah_zip_codes_geo.min.json')
zips_geo = req.json()


In [ ]:
zips_geo['features'][1]['properties']['ZCTA5CE10']

In [ ]:
zips_geo['features'][0]['properties']['ZCTA5CE10'] = '84331'
zips_geo['features'][0]['properties']['ZCTA5CE10']

In [ ]:
#needed the correct key_on address and to make sure zip data type matched the json zip data type (ints,strings) not float

req = requests.get('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ut_utah_zip_codes_geo.min.json')
zips_geo = req.json()

In [ ]:
m = folium.Map(location=[41.1097, -111.9827], zoom_start=9)

folium.Choropleth(
    geo_data=zips_geo,
    name='choropleth',
    data=zdf,
    columns=['zip', 'rate'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color = 'Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name= 'rate per 100K'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
##check to see my geojson is good, basic outlines of zip codes in UT
m = folium.Map(location=[40.8486, -111.9069], zoom_start=3)

folium.GeoJson(
    zips_geo,
    name='geojson'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
#stock example

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
#https://python-visualization.github.io/folium/quickstart.html

In [ ]:
#read in new data
c17 = pd.read_csv('covid417.csv')

In [ ]:

c17.rename(columns={"rate": "rate per 100K"})

In [ ]:
c17

In [ ]:
thermometers = 15*250

probes = (30*2000)*(100/1000)
thermometers+probes

In [ ]:
thermometers

In [ ]:
probes